In [1]:
import os
import numpy as np
import nibabel as ni

THIS SCRIPT WILL 
* SPLIT THE HIPPOCAMPUS INTO HEMISPHERES 
* REGISTER TO A WEIRD TEMPLATE 
* MOVE IT TO MNC
* MAP THE LABELS TO A HIPPOCAMPAL SURFACE. BOOYA

In [3]:
wdir = '/data1/users/jvogel/Allen_Human_Brain_Atlas/surfaces/'
tst = '/data1/users/jvogel/Allen_Human_Brain_Atlas/hipp_10_c_solution.nii'

In [46]:
project_hippocampus_labels_to_surface(tst,wdir)

no tfm entered. assuming youre in 1mm MNI space...
lateralizing...
obtaining some information from the image --  lh


/home/users/jvogel/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in greater


coregistering to 0.3mm space --  lh
converting to mnc --  lh
creating text file of surface labels --  lh
making final surface image --  lh
created new surface project file:  /data1/users/jvogel/Allen_Human_Brain_Atlas/surfaces/0p3mm_hipp_10_c_solution_lh_labeled.obj
obtaining some information from the image --  rh
coregistering to 0.3mm space --  rh
converting to mnc --  rh
creating text file of surface labels --  rh
making final surface image --  rh
created new surface project file:  /data1/users/jvogel/Allen_Human_Brain_Atlas/surfaces/0p3mm_hipp_10_c_solution_rh_labeled.obj


In [48]:
project_hippocampus_labels_to_surface('/data1/users/jvogel/Allen_Human_Brain_Atlas/hipp_5_c_solution.nii',wdir)

no tfm entered. assuming youre in 1mm MNI space...
lateralizing...


/home/users/jvogel/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in greater


obtaining some information from the image --  lh
coregistering to 0.3mm space --  lh
converting to mnc --  lh
creating text file of surface labels --  lh
making final surface image --  lh
created new surface project file:  /data1/users/jvogel/Allen_Human_Brain_Atlas/surfaces/0p3mm_hipp_5_c_solution_lh_labeled.obj
obtaining some information from the image --  rh
coregistering to 0.3mm space --  rh
converting to mnc --  rh
creating text file of surface labels --  rh
making final surface image --  rh
created new surface project file:  /data1/users/jvogel/Allen_Human_Brain_Atlas/surfaces/0p3mm_hipp_5_c_solution_rh_labeled.obj


In [47]:
# VARIABLES

def project_hippocampus_labels_to_surface(hipp, wdir = './', tfm = None, mt_command = '/opt/minc/bin/'):
    
    template = '/home/users/jvogel/Science/templates/templates/brains/nii_model.nii'
    if type(tfm) == type(None):
        print('no tfm entered. assuming youre in 1mm MNI space...')
        tfm = '/home/users/jvogel/Science/templates/tfms/maget_temp_2_MNI1mm_brain_tfm_inverted'
    l_hipp = '/home/users/jvogel/Science/templates/templates/objects/new_lefthc.obj'
    r_hipp = '/home/users/jvogel/Science/templates/templates/objects/new_righthc.obj'

    print('lateralizing...')
    nfls = lateralize(hipp,wdir)
    hs = dict(zip(['lh','rh'], nfls))
    
    for hemi, fl in hs.items():
        print('obtaining some information from the image -- ',hemi)
        maxlab = sorted(np.unique(ni.load(fl).get_data()))[-1]
        
        # define outputs [so we can pick up where we left off]
        fl03 = os.path.join(wdir,'0p3mm_%s'%os.path.split(fl)[-1])
        if '.gz' in fl03:
            mnc = '%s.mnc.gz'%fl03.split('.ni')[0]
        else:
            mnc = '%s.mnc'%fl03.split('.ni')[0]
        txt = '%s.txt'%mnc.split('.mn')[0]
        final = '%s_labeled.obj'%txt.split('.tx')[0]
        
                        
        if not os.path.isfile(fl03) and not os.path.isfile(fl03+'.gz'):
            print('coregistering to 0.3mm space -- ', hemi)
            cmd = 'fsl5.0-flirt -interp nearestneighbour -in %s -ref %s -applyxfm -init %s -out %s'%(fl, template, 
                                                                                                     tfm, fl03)
            os.system(cmd)
        if not os.path.isfile(mnc):  
            print('converting to mnc -- ', hemi)
            os.system('%snii2mnc %s %s'%(mt_command,fl03,mnc))
        
        if not os.path.isfile(txt):
            print('creating text file of surface labels -- ',hemi)
            if hemi == 'lh':
                obj = l_hipp
            else:
                obj = r_hipp
            os.system('%svolume_object_evaluate -linear %s %s %s'%(mt_command, mnc, obj, txt))
            
        if not os.path.isfile(final):
            print('making final surface image -- ',hemi)
            os.system('%scolour_object %s %s %s spectral 1 %s'%(mt_command, obj, txt, final, maxlab))
    
        print('created new surface project file: ',final)
    
def lateralize(infile, wdir):
    jnk = ni.load(infile)
    dat = jnk.get_data()
    aff = jnk.affine

    lh = np.zeros_like(dat)
    rh = np.zeros_like(dat)

    div = dat.shape[0] / 2
    
    hcoords = np.where(dat>0)
    lcs = [x for x in range(len(hcoords[0])) if hcoords[0][x] > div]
    lh[hcoords[0][lcs],hcoords[1][lcs],hcoords[2][lcs]] = dat[hcoords[0][lcs],hcoords[1][lcs],hcoords[2][lcs]]

    rcs = [x for x in range(len(hcoords[0])) if hcoords[0][x] < div]
    rh[hcoords[0][rcs],hcoords[1][rcs],hcoords[2][rcs]] = dat[hcoords[0][rcs],hcoords[1][rcs],hcoords[2][rcs]]
    
    flnm = os.path.split(infile)[-1].split('.ni')[0]
    nfls = [os.path.join(wdir,'%s_%s.nii'%(flnm,x)) for x in ['lh','rh']] 
    
    ni.Nifti1Image(lh,aff).to_filename(nfls[0])
    ni.Nifti1Image(rh,aff).to_filename(nfls[1])
    
    return nfls